Spark Data Sources
This notebook shows how to use Spark Data Sources Interface API to read file formats:

* Parquet
* JSON
* CSV
* Avro
* ORC
* Image
* Binary

A full list of DataSource methods is available here



Define paths for the various data sources

In [1]:
parquet_file = "../databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet"
json_file = "../databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
csv_file = "../databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
orc_file = "../databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
avro_file = "../databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"

In [2]:
from pyspark.sql import SparkSession
#create a SparkSession
spark = (SparkSession
    .builder
    .appName("Example-3_6")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.2")
    .getOrCreate())

## Parquet Data Source

In [3]:
df = (spark
      .read
      .format("parquet")
      .option("path", parquet_file)
      .load())

Another way to read this same data using a variation of this API

In [4]:
df2 = (spark.
       read.
       parquet(parquet_file))

In [7]:
df.show(10, False)
df2.toPandas().head(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



,DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
0,United States,Romania,1
1,United States,Ireland,264
2,United States,India,69
3,Egypt,United States,24
4,Equatorial Guinea,United States,1
5,United States,Singapore,25
6,United States,Grenada,54
7,Costa Rica,United States,477
8,Senegal,United States,29
9,United States,Marshall Islands,44


## Use SQL
This will create an unmanaged temporary view

In [38]:
spark.sql(
    """
    CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
        USING parquet
            OPTIONS (
              path "../databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet"
            )
    """
)

DataFrame[]

Use SQL to query the table

The outcome should be the same as one read into the DataFrame above

In [40]:
df = spark.sql("SELECT * FROM us_delay_flights_tbl")

In [42]:
df.show(10, truncate=False)
df.toPandas().head(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



,DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
0,United States,Romania,1
1,United States,Ireland,264
2,United States,India,69
3,Egypt,United States,24
4,Equatorial Guinea,United States,1
5,United States,Singapore,25
6,United States,Grenada,54
7,Costa Rica,United States,477
8,Senegal,United States,29
9,United States,Marshall Islands,44


## JSON Data Source

In [13]:
df = spark.read.format("json").option("path", json_file).load()

In [14]:
df.show(10, truncate=False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
|United States    |Singapore          |1    |
|United States    |Grenada            |62   |
|Costa Rica       |United States      |588  |
|Senegal          |United States      |40   |
|Moldova          |United States      |1    |
+-----------------+-------------------+-----+
only showing top 10 rows



In [15]:
df2 = spark.read.json(json_file)

In [16]:
df2.show(10, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
|United States    |Singapore          |1    |
|United States    |Grenada            |62   |
|Costa Rica       |United States      |588  |
|Senegal          |United States      |40   |
|Moldova          |United States      |1    |
+-----------------+-------------------+-----+
only showing top 10 rows



## CSV Data Source

In [17]:
df = (spark
      .read
	 .format("csv")
	 .option("header", "true")
	 .schema(schema)
	 .option("mode", "FAILFAST")  # exit if any errors
	 .option("nullValue", "")	  # replace any null data field with “”
	 .option("path", csv_file)
	 .load())

In [18]:
df.show(10, truncate = False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [19]:
(df.write.format("parquet")
  .mode("overwrite")
  .option("path", "/tmp/data/parquet/df_parquet")
  .option("compression", "snappy")
  .save())

In [20]:
!fs ls /tmp/data/parquet/df_parquet

/bin/bash: fs: orden no encontrada


In [21]:
df2 = (spark
       .read
       .option("header", "true")
       .option("mode", "FAILFAST")	 # exit if any errors
       .option("nullValue", "")
       .schema(schema)
       .csv(csv_file))

In [22]:
df2.show(10, truncate=False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



## ORC Data Source

In [23]:
df = (spark.read
      .format("orc")
      .option("path", orc_file)
      .load())

In [24]:
df.show(10, truncate=False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



## Avro Data Source

In [25]:
df = (spark.read
      .format("avro")
      .option("path", avro_file)
      .load())

In [26]:
df.show(10, truncate=False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



## Image

In [27]:
from pyspark.ml import image

image_dir = "../databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()

images_df.select("image.height", "image.width", "image.nChannels", "image.mode", "label").show(5, truncate=False)

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)
 |-- label: integer (nullable = true)

+------+-----+---------+----+-----+
|height|width|nChannels|mode|label|
+------+-----+---------+----+-----+
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |1    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
+------+-----+---------+----+-----+
only showing top 5 rows



## Binary

In [28]:
path = "../databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
binary_files_df = (spark.read.format("binaryFile")
  .option("pathGlobFilter", "*.jpg")
  .load(path))

binary_files_df.show(5)

+--------------------+-------------------+------+--------------------+-----+
|                path|   modificationTime|length|             content|label|
+--------------------+-------------------+------+--------------------+-----+
|file:/media/jose/...|2021-04-15 02:34:17| 55037|[FF D8 FF E0 00 1...|    0|
|file:/media/jose/...|2021-04-15 02:34:17| 54634|[FF D8 FF E0 00 1...|    1|
|file:/media/jose/...|2021-04-15 02:34:17| 54624|[FF D8 FF E0 00 1...|    0|
|file:/media/jose/...|2021-04-15 02:34:17| 54505|[FF D8 FF E0 00 1...|    0|
|file:/media/jose/...|2021-04-15 02:34:17| 54475|[FF D8 FF E0 00 1...|    0|
+--------------------+-------------------+------+--------------------+-----+
only showing top 5 rows



To ignore any partitioning data discovery in a directory, you can set the recursiveFileLookup to true.

In [29]:
binary_files_df = (spark.read.format("binaryFile")
   .option("pathGlobFilter", "*.jpg")
   .option("recursiveFileLookup", "true")
   .load(path))
binary_files_df.show(5)

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|file:/media/jose/...|2021-04-15 02:34:17| 55037|[FF D8 FF E0 00 1...|
|file:/media/jose/...|2021-04-15 02:34:17| 54634|[FF D8 FF E0 00 1...|
|file:/media/jose/...|2021-04-15 02:34:17| 54624|[FF D8 FF E0 00 1...|
|file:/media/jose/...|2021-04-15 02:34:17| 54505|[FF D8 FF E0 00 1...|
|file:/media/jose/...|2021-04-15 02:34:17| 54475|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

